**Install and Import IndicLID**

In [ ]:
!pip3 install fasttext
!pip3 install transformers

In [ ]:
!git clone https://github.com/AI4Bharat/IndicLID.git

In [ ]:
%cd "/content/IndicLID/Inference"

In [ ]:
%mkdir models
%cd "/content/IndicLID/Inference/models"

In [ ]:
!wget https://github.com/AI4Bharat/IndicLID/releases/download/v1.0/indiclid-bert.zip
!wget https://github.com/AI4Bharat/IndicLID/releases/download/v1.0/indiclid-ftn.zip
!wget https://github.com/AI4Bharat/IndicLID/releases/download/v1.0/indiclid-ftr.zip

In [ ]:
!unzip indiclid-bert.zip
!unzip indiclid-ftn.zip
!unzip indiclid-ftr.zip

In [ ]:
# %cd "/content/IndicLID/"
%cd "/content/IndicLID/Inference"

In [ ]:
from ai4bharat.IndicLID import IndicLID

IndicLID_model = IndicLID(input_threshold = 0.5, roman_lid_threshold = 0.6)

Extract Posts Body along with Replies

In [9]:
# Import necessary libraries
import json
from typing import List, Tuple

# Function to extract all 'body' fields from comments and their replies
def extract_bodies(data: List[dict]) -> List[str]:
    """
    Extracts 'body' fields from comments and their replies.

    Args:
        data (List[dict]): List of dictionaries containing comment data.

    Returns:
        List[str]: List of extracted 'body' fields.
    """
    bodies: List[str] = []
    for item in data:  # Assuming the top-level structure is a list
        if 'comments' in item:  # Check if the 'comments' key exists
            for comment in item['comments']:
                if 'body' in comment:  # Check if the 'body' key exists in the comment
                    clean_body = comment['body'].replace('\n', ' ')  # Replace newline characters with a space
                    bodies.append(clean_body)
                if 'reply' in comment:  # Check if there are replies
                    for reply in comment['reply']:
                        if 'body' in reply:  # Check if the 'body' key exists in the reply
                            clean_reply_body = reply['body'].replace('\n', ' ')  # Similarly, clean up the reply body
                            bodies.append(clean_reply_body)
    return bodies

# Function to read JSON data from a file and extract bodies
def read_and_extract(file_path: str) -> List[str]:
    """
    Reads JSON data from a file and extracts 'body' fields.

    Args:
        file_path (str): Path to the JSON file.

    Returns:
        List[str]: List of extracted 'body' fields.
    """
    with open(file_path, 'r', encoding='utf-8') as file:  # Open the file in read mode
        data: List[dict] = json.load(file)  # Load JSON data from the file
        return extract_bodies(data)  # Extract bodies using the function defined earlier

# Specify the path to your JSON file
file_path = '/content/Delhi-9.json'  # Update this to the path of your JSON file
comment_bodies: List[str] = read_and_extract(file_path)
#print(comment_bodies)
test_samples1 = comment_bodies
seen = set()
unique_list = []
for item in test_samples1:
    if item not in seen:
        unique_list.append(item)
        seen.add(item)


Unit test, to see if the data has been read correctly and model is predicting correctly


In [10]:
test_samples = comment_bodies[:3]

In [ ]:
batch_size = 1
outputs1 = IndicLID_model.batch_predict(test_samples, batch_size)
print(outputs1)

Runs on the entire datset provided only the unique rows

In [12]:
batch_size = 1
outputs = IndicLID_model.batch_predict(unique_list, batch_size)

In [ ]:
print(outputs)

Prints the total number of data

In [14]:
#english_list = []
count=0
for item in outputs:
    #if item[1] == 'eng_Latn':
        #print(item[0])
        #english_list.append(item)
        count = count + 1
print(count)

4986


In [ ]:
import unittest
from unittest.mock import MagicMock

class TestModelPredictions(unittest.TestCase):
    def setUp(self):
        """Setting up mocked model"""
        self.mocked_model = MagicMock()
        self.mocked_model.batch_predict.return_value = [
            ("I'll be landing in delhi tomorrow, will update it :')", 'eng_Latn', 1.0000157, 'IndicLID-FTR'),
            ("Mast video banake upload karna bhai, muje bhi dekhna hai pollution", 'ben_Latn', 0.60675067, 'IndicLID-FTR'),
            ("Window seat confirm mili hai ?", 'eng_Latn', 0.990012, 'IndicLID-FTR')
        ]
        self.test_samples = [
            "I'll be landing in delhi tomorrow, will update it :')",
            "Mast video banake upload karna bhai, muje bhi dekhna hai pollution",
            "Window seat confirm mili hai ?"
        ]
        self.batch_size = 1

    def test_batch_predictions(self):
        """Test the batch prediction output structure and data types."""
        outputs = self.mocked_model.batch_predict(self.test_samples, self.batch_size)

        # Check if the output is a list
        self.assertIsInstance(outputs, list, "Output should be a list.")

        # Check each item in the output list
        for item in outputs:
            self.assertIsInstance(item, tuple, "Every output item must be a tuple.")
            self.assertEqual(len(item), 4, "Every tuple must contain exactly four elements.")

            # Check data types
            self.assertIsInstance(item[0], str, "First element of the tuple must be a string.")
            self.assertIsInstance(item[1], str, "Second element of the tuple must be a string.")
            self.assertIsInstance(item[2], float, "Third element of the tuple must be a float.")
            self.assertIsInstance(item[3], str, "Fourth element of the tuple must be a string.")

# Running the tests
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
